# Exercise 4 - Reflection pattern implementation

In [ ]:
# set your open ai key in .env file (text file) as OPENAI_API_KEY=your-key
# your .env file should be in tthe same directory as this file
from dotenv import load_dotenv
_ = load_dotenv()

In [ ]:
import os
from typing import TypedDict, List
from langgraph.graph import StateGraph, END, START
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI

# --- Setup LLM (Assumes you have OPENAI_API_KEY set) ---
# You can swap this for Anthropic, Llama, etc.
llm = ChatOpenAI(model="gpt-4o", temperature=0.7)

# --- 1. Define the State ---
class WritingState(TypedDict):
    topic: str
    draft: str
    critique: str
    revision_count: int

# --- 2. Define the PROMPTS (The "Brains") ---

WRITER_PROMPT = """
You are a technical content writer. 
If this is the first draft, write a short, engaging blog post about the topic: {topic}.
If you have received feedback, rewrite the draft to address the critique below.

Current Critique: {critique}
"""

# This is the "Reflector" logic you asked for
REFLECTOR_PROMPT = """
You are a Senior Technical Editor. Your job is to critique the following blog post.
You must be strict but helpful. 

Check for:
1. Technical Accuracy (Are the terms used correctly?)
2. Clarity (Is it easy to read?)
3. Engagement (Is it boring?)

If the post is excellent and ready to publish, reply with exactly one word: FINISH.
Otherwise, provide a numbered list of specific actionable changes the writer should make.
"""

# --- 3. Define the Nodes ---

def writer_node(state: WritingState):
    print(f"\n--- WRITER: Working on draft {state['revision_count'] + 1} ---")
    
    # Construct the message history
    messages = [
        SystemMessage(content=WRITER_PROMPT.format(topic=state['topic'], critique=state.get('critique', 'None'))),
        HumanMessage(content=state.get('draft', f"Write a blog about {state['topic']}"))
    ]
    
    response = llm.invoke(messages)
    
    return {
        "draft": response.content, 
        "revision_count": state["revision_count"] + 1
    }

def reflector_node(state: WritingState):
    print("\n--- REFLECTOR: Reviewing draft... ---")
    
    messages = [
        SystemMessage(content=REFLECTOR_PROMPT),
        HumanMessage(content=state['draft'])
    ]
    
    response = llm.invoke(messages)
    
    print(f"Reflector Feedback: {response.content[:100]}...") # Print preview of feedback
    return {"critique": response.content}

# --- 4. Define the Logic (The Router) ---

def should_continue(state: WritingState):
    critique = state["critique"]
    revision_count = state["revision_count"]
    
    # Stop if the editor is happy OR if we've tried 3 times (safety valve)
    if "FINISH" in critique or revision_count >= 3:
        return "end"
    else:
        return "writer" # Loop back

# --- 5. Build the Graph ---

builder = StateGraph(WritingState)

builder.add_node("writer", writer_node)
builder.add_node("reflector", reflector_node)

builder.add_edge(START, "writer")
builder.add_edge("writer", "reflector")

builder.add_conditional_edges(
    "reflector",
    should_continue,
    {
        "writer": "writer",  # The Loop
        "end": END
    }
)

app = builder.compile()

# --- 6. Run it ---
print(">>> STARTING BLOG AGENT")
initial_state = {
    "topic": "The importance of 'Reflection' in Agentic AI",
    "draft": "",
    "critique": "",
    "revision_count": 0
}

# This will loop until the Reflector says "FINISH"
final_output = app.invoke(initial_state)

print("\n\n>>> FINAL RESULT <<<")
print(final_output["draft"])